In [80]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

In [81]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from clustering_utils import cluster_position_group
from plot_utils import plot_archetypes_radar

df = pd.read_csv("../data/fifa23.csv", sep=';', encoding='utf-8')
df_field_players = df[df['Best Position'].str.contains('GK') == False]

ImportError: cannot import name 'plot_archetypes_radar' from 'plot_utils' (c:\Users\rgbej\OneDrive\Escritorio\Aprender Codigo\Arquetipos\fifa23-archetypes\src\plot_utils.py)

In [ ]:
stats = ['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
    'Dribbling', 'Curve', 'Freekick Accuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision',
    'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle']

df_field_players = df_field_players.copy()

# Ensure the columns you’re modifying are floats
df_field_players[stats] = df_field_players[stats].astype(float)

# Divide all columns in one vectorized operation
df_field_players.loc[:, stats] = df_field_players[stats].div(df['Overall'], axis=0)

features = [
    'Height(in cm)', 'Weight(in kg)',
    'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
    'Dribbling', 'Curve', 'Freekick Accuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision',
    'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle'
]

In [ ]:
position_groups = {
    'CB': 'DEF', 'LB': 'DEF', 'RB': 'DEF', 'LWB': 'DEF', 'RWB': 'DEF',
    'CDM': 'MID', 'CM': 'MID', 'CAM': 'MID', 'LM': 'MID', 'RM': 'MID',
    'ST': 'FWD', 'CF': 'FWD', 'LW': 'FWD', 'RW': 'FWD'
}

df_field_players['Position Group'] = df_field_players['Best Position'].map(position_groups)

## Defensas

In [ ]:
defense_features = [
    'Height(in cm)', 'Weight(in kg)',
    'Crossing', 'Heading Accuracy', 'Short Passing',
    'Dribbling', 'LongPassing', 'BallControl',
    'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Jumping', 'Stamina', 'Strength',
    'Aggression', 'Interceptions', 'Positioning',
    'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle'
]

df_defenders, kmeans_def = cluster_position_group(
    df_field_players, 'DEF', defense_features, n_clusters=5
)

cluster_means_def = df_defenders.groupby('Cluster')[defense_features].mean()
cluster_means_def.to_csv("../outputs/cluster_means/clusters_defense.csv")
print(cluster_means_def)


         Height(in cm)  Weight(in kg)  Crossing  Heading Accuracy  \
Cluster                                                             
0           178.035311      72.254237  0.962854          0.821792   
1           189.242583      82.817627  0.561646          1.013613   
2           184.667279      77.974877  0.770992          0.971291   
3           185.965585      78.388397  0.558754          0.975089   
4           177.227872      71.099812  0.920745          0.799626   

         Short Passing  Dribbling  LongPassing  BallControl  Acceleration  \
Cluster                                                                     
0             0.951017   0.946979     0.866681     0.956063      1.072432   
1             0.899877   0.670783     0.828714     0.828877      0.720240   
2             0.958911   0.838596     0.896890     0.918224      0.953113   
3             0.814273   0.622873     0.686290     0.756884      1.005005   
4             0.884398   0.930434     0.776009     0.9

In [ ]:
archetype_names_def = {
    0: "Offensive Full-Back",
    1: "Physical Tower CB",
    2: "Ball Playing CB",
    3: "Aggresive Fast CB",
    4: "Attacking Wing-Back"
}

df_defenders['archetype'] = df_defenders['Cluster'].map(archetype_names_def)

for archetype, group in df_defenders.groupby('archetype'):
    print(f"\n🔹 {archetype}")
    display(group[['Known As', 'Best Position', 'Overall', 'Height(in cm)']].head(5))


🔹 Aggresive Fast CB


,Known As,Best Position,Overall,Height(in cm)
308,Heberto Gaúcho,CB,81,181
543,Ibañez,CB,79,185
844,J. Aidoo,CB,78,184
1269,Antonaldinho,CB,76,188
1319,Igor,CB,76,185



🔹 Attacking Wing-Back


,Known As,Best Position,Overall,Height(in cm)
1209,L. Advíncula,RB,76,180
1851,R. Henry,LWB,75,170
1926,Luis Pérez,RB,74,174
2055,R. Cannon,RWB,74,180
2074,F. Sacko,RB,74,179



🔹 Ball Playing CB


,Known As,Best Position,Overall,Height(in cm)
25,Marquinhos,CB,88,183
46,D. Alaba,CB,86,180
91,L. Hernández,CB,84,184
129,J. Koundé,CB,84,181
133,Éder Militão,CB,84,186



🔹 Offensive Full-Back


,Known As,Best Position,Overall,Height(in cm)
24,João Cancelo,LB,88,182
39,A. Robertson,LB,87,178
69,K. Walker,RB,85,183
71,Jordi Alba,LB,85,170
77,M. Acuña,LB,85,172



🔹 Physical Tower CB


,Known As,Best Position,Overall,Height(in cm)
9,V. van Dijk,CB,90,193
20,Rúben Dias,CB,88,187
30,A. Rüdiger,CB,87,190
40,K. Koulibaly,CB,87,187
50,Thiago Silva,CB,86,183


In [ ]:
radar_features = [f for f in defense_features if f not in ['Height(in cm)', 'Weight(in kg)']]

plot_archetypes_radar(
    cluster_means=cluster_means_def,
    radar_features=radar_features,
    archetype_names=archetype_names_def,
    position_group='DEF'
)

## Mediocentros


In [ ]:
midfield_features = [
    'Height(in cm)', 'Weight(in kg)',
    'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
    'Dribbling','Curve','Freekick Accuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Jumping', 'Stamina', 'Strength','Long Shots',
    'Aggression', 'Interceptions', 'Positioning', 'Vision',
    'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle'
]

df_midfielders, kmeans_def = cluster_position_group(
    df_field_players, 'MID', midfield_features, n_clusters=5
)
cluster_means_mid = df_midfielders.groupby('Cluster')[midfield_features].mean()
cluster_means_mid.to_csv("../outputs/cluster_means/clusters_midfield.csv")
print(cluster_means_mid)


         Height(in cm)  Weight(in kg)  Crossing  Finishing  Heading Accuracy  \
Cluster                                                                        
0           181.160889      75.130804  0.918524   0.869726          0.823792   
1           175.514957      69.266382  0.906118   0.915162          0.686120   
2           181.272081      74.837563  0.752762   0.675077          0.856674   
3           176.096467      69.503085  0.863460   0.818207          0.781583   
4           176.794399      70.759027  0.933574   0.930306          0.709247   

         Short Passing   Volleys  Dribbling     Curve  Freekick Accuracy  ...  \
Cluster                                                                   ...   
0             1.033834  0.818292   0.985538  0.940150           0.891473  ...   
1             0.938086  0.786258   1.049622  0.850245           0.737577  ...   
2             1.025079  0.601332   0.906574  0.710399           0.684608  ...   
3             1.041718  0.727513  

In [ ]:
archetype_names_mid = {
    0: "Box-to-Box Midfielder",
    1: "Attacking Wide Midfielder",
    2: "Holding Midfielder",
    3: "Mobile Playmaker",
    4: "Attacking Midfielder"
}

df_midfielders['archetype'] = df_midfielders['Cluster'].map(archetype_names_mid)

for archetype, group in df_midfielders.groupby('archetype'):
    print(f"\n🔹 {archetype}")
    display(group[['Known As', 'Best Position', 'Overall', 'Short Passing']].head(5))


🔹 Attacking Midfielder


,Known As,Best Position,Overall,Short Passing
0,L. Messi,CAM,91,1.000000
15,S. Mané,LM,89,0.943820
23,Bernardo Silva,CAM,88,1.011364
49,P. Dybala,CAM,86,1.000000
56,K. Coman,LM,86,0.930233



🔹 Attacking Wide Midfielder


,Known As,Best Position,Overall,Short Passing
126,M. Diaby,RM,84,0.952381
244,C. Pulisic,RM,82,0.939024
369,A. Saint-Maximin,CAM,81,0.938272
415,N. González,LM,80,1.000000
537,S. Chukwueze,LM,79,0.924051



🔹 Box-to-Box Midfielder


,Known As,Best Position,Overall,Short Passing
3,K. De Bruyne,CM,91,1.021978
17,J. Kimmich,CDM,89,0.977528
28,T. Kroos,CM,88,1.056818
29,L. Modrić,CM,88,1.034091
34,T. Müller,CAM,87,0.977011



🔹 Holding Midfielder


,Known As,Best Position,Overall,Short Passing
13,Casemiro,CDM,89,0.943820
19,N. Kanté,CDM,89,0.921348
31,Rodri,CDM,87,0.988506
32,Fabinho,CDM,87,0.965517
72,Sergio Busquets,CDM,85,1.023529



🔹 Mobile Playmaker


,Known As,Best Position,Overall,Short Passing
36,M. Verratti,CM,87,1.034483
82,Pedri,CM,85,1.023529
141,A. Davies,LM,84,0.976190
233,N. Mazraoui,RM,82,1.012195
360,S. Lobotka,CM,81,1.049383


In [ ]:
radar_features = [f for f in midfield_features if f not in ['Height(in cm)', 'Weight(in kg)']]

plot_archetypes_radar(
    cluster_means=cluster_means_mid,
    radar_features=radar_features,
    archetype_names=archetype_names_mid,
    position_group='MID'
)

## Delanteros

In [ ]:
forward_features = [
    'Height(in cm)', 'Weight(in kg)',
    'Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys',
    'Dribbling','Curve','Freekick Accuracy', 'LongPassing', 'BallControl',
    'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
    'Jumping', 'Stamina', 'Strength','Long Shots',
    'Positioning', 'Vision'
]

df_forwards, kmeans_def = cluster_position_group(
    df_field_players, 'FWD', forward_features, n_clusters=5
)

cluster_means_fwd = df_forwards.groupby('Cluster')[forward_features].mean()
cluster_means_fwd.to_csv("../outputs/cluster_means/clusters_forward.csv")
print(cluster_means_fwd)


         Height(in cm)  Weight(in kg)  Crossing  Finishing  Heading Accuracy  \
Cluster                                                                        
0           181.418014      75.864896  0.857791   1.010876          0.944624   
1           179.650206      73.240055  0.753215   1.013421          0.914920   
2           174.902010      68.939698  0.928121   0.947714          0.700773   
3           188.990268      83.839416  0.677949   1.033354          1.059595   
4           185.655451      79.188425  0.602920   1.035239          1.017969   

         Short Passing   Volleys  Dribbling     Curve  Freekick Accuracy  ...  \
Cluster                                                                   ...   
0             0.952179  0.935132   0.997504  0.889948           0.814673  ...   
1             0.884019  0.847351   1.009450  0.772804           0.655410  ...   
2             0.943200  0.852043   1.052324  0.916089           0.810812  ...   
3             0.913018  0.953272  

In [ ]:
archetype_names_fwd = {
    0: "Complete Forward",
    1: "Second Striker",
    2: "Explosive Winger",
    3: "Target Man",
    4: "Poacher"
}

df_forwards['archetype'] = df_forwards['Cluster'].map(archetype_names_fwd)

for archetype, group in df_forwards.groupby('archetype'):
    print(f"\n🔹 {archetype}")
    display(group[['Known As', 'Best Position', 'Overall', 'Short Passing']].head(5))


🔹 Complete Forward


,Known As,Best Position,Overall,Short Passing
1,K. Benzema,CF,91,0.978022
4,K. Mbappé,ST,91,0.934066
5,M. Salah,RW,90,0.933333
12,H. Son,LW,89,0.943820
59,L. Martínez,ST,86,0.883721



🔹 Explosive Winger


,Known As,Best Position,Overall,Short Passing
11,Neymar Jr,LW,89,0.955056
47,R. Sterling,LW,86,0.965116
48,R. Mahrez,RW,86,0.965116
63,Vinícius Jr.,LW,86,0.872093
95,D. Mertens,CF,84,1.000000



🔹 Poacher


,Known As,Best Position,Overall,Short Passing
166,V. Osimhen,ST,83,0.867470
474,D. Calvert-Lewin,ST,80,0.887500
494,A. Belotti,ST,80,0.850000
539,Beto,ST,79,0.848101
545,B. Dia,ST,79,0.936709



🔹 Second Striker


,Known As,Best Position,Overall,Short Passing
754,J. Martínez,ST,78,0.858974
1009,P. Daka,ST,77,0.883117
1230,E. Dennis,ST,76,0.855263
1258,E. Boateng,ST,76,0.868421
1460,A. Elis,ST,75,0.866667



🔹 Target Man


,Known As,Best Position,Overall,Short Passing
2,R. Lewandowski,ST,91,0.923077
8,Cristiano Ronaldo,ST,90,0.888889
10,H. Kane,ST,89,0.943820
21,E. Haaland,ST,88,0.840909
44,C. Immobile,ST,86,0.918605


In [ ]:
radar_features = [f for f in forward_features if f not in ['Height(in cm)', 'Weight(in kg)']]

plot_archetypes_radar(
    cluster_means=cluster_means_fwd,
    radar_features=radar_features,
    archetype_names=archetype_names_fwd,
    position_group='FWD'
)